In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8000),
    tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8000)])

logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

logical_gpus  

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPU, 2 Logical GPUs


[LogicalDevice(name='/device:GPU:0', device_type='GPU'),
 LogicalDevice(name='/device:GPU:1', device_type='GPU')]

In [9]:
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import GraphDataLoader

In [38]:
train = np.array([[1,1,2], [1,1,3],[1,1,4],[5,1,3],[5,1,4],[6,1,3],[6,1,2],[6,1,4],[6,1,7]])
test = np.array([[3,1,2], [4,1,3],[5,1,4], [5,1,2], [1,1,5]])
val = np.array([[3,1,6], [2,1,2],[1,1,6]])
use_filter = {'train':train,'test':test, 'val':val}

# complementary objects to 1,1,?: train: 2, 3, 4, test: 5, val: 6 
# complementary subjects to ?,1,2: train: 1, 6, test: 3, 5, val: 2

data = GraphDataLoader(train, batch_size=1, dataset_type="train", use_filter=use_filter)

In [39]:
triple = np.array([[0,0,1]])
converted = data.backend.mapper.get_indexes(triple, order='ind2raw')
print(converted)
for batch in data:
    print("Complementary objects to ({}, {}, ?)".format(converted[0,0],converted[0,1]))
    print(data.backend.mapper.get_indexes(data.get_complementary_objects(triple, use_filter=use_filter)[0], type_of='e', order='ind2raw'))
    print("Complementary subjects to (?, {}, {})".format(converted[0,1],converted[0,2]))    
    print(data.backend.mapper.get_indexes(data.get_complementary_subjects(triple, use_filter=use_filter)[0], type_of='e', order='ind2raw'))
    print("Raw: ", data.get_complementary_entities(triple, use_filter=use_filter))    
    break

[[1 1 2]]
Complementary objects to (1, 1, ?)
['2' '3' '4' '5' '6']
Complementary subjects to (?, 1, 2)
['1' '6' '3' '5' '2']
Raw:  ([[0, 5, 2, 4, 1]], [[1, 2, 3, 4, 5]])
